In [1]:
import pandas as pd
df = pd.read_excel("CeaData.xlsx")
df

,Role,Experience,Performance,Internet,Location,Approval
0,Technical,Senior,Excellent,Good,Urban,Yes
1,Technical,Junior,Excellent,Good,Urban,Yes
2,Non-Technical,Junior,Average,Poor,Rural,No
3,Technical,Senior,Average,Good,Rural,No
4,Technical,Senior,Excellent,Good,Rural,Yes


In [2]:
X = df.drop("Approval",axis=1)
y = df["Approval"]
columns = df.columns.tolist()
print(columns)

['Role', 'Experience', 'Performance', 'Internet', 'Location', 'Approval']


In [3]:
num_attributes = len(columns) - 1
S = ["Ø"] * num_attributes
G = [["?"] * num_attributes]
print("S0:", S)
print("G0:", G)

S0: ['Ø', 'Ø', 'Ø', 'Ø', 'Ø']
G0: [['?', '?', '?', '?', '?']]


In [4]:
def is_consistent(h, x):
    return all(h[i] == '?' or h[i] == x[i] for i in range(len(h)))

def more_general(h1, h2):
    return all(h1[i] == '?' or h1[i] == h2[i] for i in range(len(h1)))


In [5]:
S = ['Ø'] * num_attributes
G = [['?'] * num_attributes]

for index, row in df.iterrows():
    instance = row.iloc[:-1].tolist()
    label = row.iloc[-1]

    print(f"\nExample {index+1}: {instance} → {label}")

    if label == "Yes":
        for i in range(num_attributes):
            if S[i] == 'Ø':
                S[i] = instance[i]
            elif S[i] != instance[i]:
                S[i] = '?'

        G = [g for g in G if is_consistent(g, instance)]

    else:
        G_new = []
        for g in G:
            if is_consistent(g, instance):
                for i in range(num_attributes):
                    if g[i] == '?' and S[i] != '?' and S[i] != instance[i]:
                        h = g.copy()
                        h[i] = S[i]
                        G_new.append(h)
            else:
                G_new.append(g)
        G = G_new

    G = [
        g for g in G
        if not any(more_general(g2, g) and g2 != g for g2 in G)
    ]

    print("S:", S)
    print("G:", G)



Example 1: ['Technical', 'Senior', 'Excellent', 'Good', 'Urban'] → Yes
S: ['Technical', 'Senior', 'Excellent', 'Good', 'Urban']
G: [['?', '?', '?', '?', '?']]

Example 2: ['Technical', 'Junior', 'Excellent', 'Good', 'Urban'] → Yes
S: ['Technical', '?', 'Excellent', 'Good', 'Urban']
G: [['?', '?', '?', '?', '?']]

Example 3: ['Non-Technical', 'Junior', 'Average', 'Poor', 'Rural'] → No
S: ['Technical', '?', 'Excellent', 'Good', 'Urban']
G: [['Technical', '?', '?', '?', '?'], ['?', '?', 'Excellent', '?', '?'], ['?', '?', '?', 'Good', '?'], ['?', '?', '?', '?', 'Urban']]

Example 4: ['Technical', 'Senior', 'Average', 'Good', 'Rural'] → No
S: ['Technical', '?', 'Excellent', 'Good', 'Urban']
G: [['?', '?', 'Excellent', '?', '?'], ['?', '?', '?', '?', 'Urban']]

Example 5: ['Technical', 'Senior', 'Excellent', 'Good', 'Rural'] → Yes
S: ['Technical', '?', 'Excellent', 'Good', '?']
G: [['?', '?', 'Excellent', '?', '?']]


In [6]:
print("\nFinal Specific Boundary (S):", S)
print("Final General Boundary (G):", G)


Final Specific Boundary (S): ['Technical', '?', 'Excellent', 'Good', '?']
Final General Boundary (G): [['?', '?', 'Excellent', '?', '?']]


In [7]:
def predict(instance, S, G):
    s_match = is_consistent(S, instance)
    g_match = all(is_consistent(g, instance) for g in G)

    if s_match and g_match:
        return "Yes"
    elif not g_match:
        return "No"
    else:
        return "Uncertain"
new_instance = ["Technical", "Senior", "Excellent", "Good", "Urban"]
prediction = predict(new_instance, S, G)
print("Prediction for new instance:", prediction)


Prediction for new instance: Yes
